# Analysis for Differential Jet Mass in Z+Jets events with NanoAODv9

In [1]:
import awkward as ak
import numpy as np
import time
import coffea
import uproot
import hist
import vector
print("awkward version ", ak.__version__)
print("coffea version ", coffea.__version__)
from coffea import util, processor
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema, BaseSchema
from collections import defaultdict
import pickle
from distributed.diagnostics.plugin import UploadDirectory
import os

awkward version  1.7.0
coffea version  0.7.11


### Automatically reload any modules used

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from smp_utils import *
from cms_utils import *
from response_maker_nanov9_lib import *
from response_maker_nanov9 import *

In [ ]:
f = uproot.open("/mnt/data/cms/store/mc/RunIISummer20UL16NanoAODv9/DYJetsToLL_M-50_HT-100to200_TuneCP5_PSweights_13TeV-madgraphMLM-pythia8/NANOAODSIM/106X_mcRun2_asymptotic_v17-v2/260000/18D0A087-30BD-FE4E-B447-5F493C2D2794.root")

In [ ]:
f["Events"].keys()

In [ ]:
from dask.distributed import Client

client = Client("tls://rappoccio-40gmail-2ecom.dask.cmsaf-prod.flatiron.hollandhpc.org:8786")
client

In [ ]:
client.register_worker_plugin(UploadDirectory("/home/cms-jovyan/QJetMassUproot", restart=True, update_path=True), nanny=True)
print(client.run(os.listdir, "dask-worker-space") )

### If desired, uncomment these to move or remove old pkl files

In [ ]:
#! mv qjetmass_zjets_gen.pkl qjetmass_zjets_gen_old.pkl
#! mv qjetmass_zjets_reco.pkl qjetmass_zjets_reco_old.pkl

In [ ]:
client.scheduler_info()['workers']

### If you want to rerun, here is the line. Otherwise keep it commented to just read the pkl file. 

In [ ]:
response_maker_nanov9(testing=False, do_gen=False, client=None, prependstr="/mnt/data/cms")
#response_maker_nanov9(testing=False, do_gen=False, client=client)

Processing:   0%|          | 0/5601 [00:00<?, ?chunk/s]

IOV IOV IOV IOV IOV IOV IOV  IOV     2016 2016 2016 20162016 2016  2016 2016 , era  , era  , era , era  , era , era   , era  , era    Run2018D Run2018DRun2018DRun2018DRun2018D
Run2018D 



Run2018D
Run2018D

IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , 

IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 IOV , era   Run2018D2016
 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , era  Run2018D
IOV  2016 , 

IOV  2016 , era  Run2018C
IOV  2016 , era  Run2018C
IOV  2016 , era  Run2018C
IOV  2016 , era  Run2018C
IOV  2016 , era  Run2018C
IOV  2016 , era  Run2018C
IOV  2016 , era  Run2018C
IOV  2016 , era  Run2018C
IOV  2016 , era  Run2018B
IOV  2016 , era  Run2018B
IOV  2016 , era  Run2018B
IOV  2016 , era  Run2018B
IOV  2016 , era  Run2018B
IOV  2016 , era  Run2018B
IOV  2016 , era  Run2018B
IOV  2016 , era  Run2018B
IOV  2016 , era  Run2018B
IOV  2016 , era  Run2018B
IOV  2016 , era  Run2018B
IOV  2016 , era  Run2018B
IOV  2016 , era  Run2018B
IOV  2016 , era  Run2018B
IOV  2016 , era  Run2018B
IOV  2016 , era  Run2018B
IOV  2016 , era  Run2018B
IOV  2016 , era  Run2018B
IOV  2016 , era  Run2018B
IOV  2016 , era  Run2018B
IOV  2016 , era  Run2018B
IOV  2016 , era  Run2018B
IOV  2016 , era  Run2018B
IOV  2016 , era  Run2018B
IOV  2016 , era  Run2018B
IOV  2016 , era  Run2018B
IOV  2016 , era  Run2018B
IOV  2016 IOV , era   2016Run2018B 
, era  Run2018B
IOV  2016 , era  Run2018B
IOV  2016 , 

In [ ]:
#response_maker_nanov9(testing=True, do_gen=True, client=client)
response_maker_nanov9(testing=True, do_gen=True, client=None, prependstr="/mnt/data/cms")

In [ ]:
#response_maker_nanov9(testing=True, do_gen=False, client=None, prependstr="/mnt/data/cms")

### Open pkl file for the output histograms

In [ ]:
with open("qjetmass_zjets_gen.pkl", "rb") as f:
    output = pickle.load( f )

In [ ]:
output.keys()

In [ ]:
output["cutflow"]